# Stream with using object detection
## Import相關套件

In [7]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import scipy.misc
import pylab


## 使用OpenCV模組，並讀取影像

In [8]:
# 加入 OpenCV 模組
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

#if tf.__version__ != '1.4.0':
#  raise ImportError('Please upgrade your tensorflow installation to v1.4.0!')

# 建立 VideoCapture 物件
cap = cv2.VideoCapture('chaplin.mp4')

fps = cap.get(cv2.CAP_PROP_FRAME_COUNT) 
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) 

output = cv2.VideoWriter('chaplin_out.mp4', cv2.VideoWriter_fourcc(*'X264'), fps, size)


# 設定擷取的畫面解析度
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

sys.path.append("..")

from utils import label_map_util
from utils import visualization_utils as vis_util


## 載入事先訓練好的模型

In [9]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [10]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # 使用無窮迴圈，持續擷取網路攝影機影像
    
    while (cap.isOpened()):
      # 讀取一個影格
      ret, image_np = cap.read()
      if ret == True: 
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        image_np_expanded = np.expand_dims(image_np, axis=0)

        (boxes, scores, classes, num) = sess.run(
             [detection_boxes, detection_scores, detection_classes, num_detections],
             feed_dict={image_tensor: image_np_expanded})

        vis_util.visualize_boxes_and_labels_on_image_array(
             image_np,
             np.squeeze(boxes),
             np.squeeze(classes).astype(np.int32),
             np.squeeze(scores),
             category_index,
             use_normalized_coordinates=True,
             line_thickness=4)
      # 以 OpenCV 視窗即時顯示辨識結果
        output.write(image_np)
      else: 
        break 
    cap.release() 
    output.release()
      #cv2.imshow('object detection', image_np)
      #if cv2.waitKey(25) & 0xFF == ord('q'):
      #  cv2.destroyAllWindows()
      #  break

[Caption](chaplin.mp4)

[Caption](chaplin_out.mp4)